In [30]:
import pandas as pd
import numpy as np
from data_science_tools.modeling import BaseUnsupervisedClassifierPandasOnly, ModelState, CapePipeline, BaseClassifierPandasOnly
from data_science_tools.taxonomies import Taxonomy
from transmo.transformers import FeatureSelect
from transmo.models import PassthroughPrediction
from sklearn.base import BaseEstimator, ClassifierMixin


from typing import Collection

In [92]:
class UnderwritingRules(BaseEstimator, ClassifierMixin):

    def __init__(self, taxonomy_labels=None):
        self.taxonomy_labels = taxonomy_labels
        super().__init__()

    @staticmethod
    def underwriting_flag_rules(cape_roof_condition_rating: str,
                                cape_roof_condition_rating_severe: float,
                                cape_roof_condition_rating_poor: float,
                                cape_roof_condition_reasons: Collection[str],
                                cape_roof_material_degradation_reason: str,
                                cape_roof_material_degradation: str):
        if (cape_roof_condition_rating
                == '-2') or (cape_roof_condition_rating == '-1' and
                             (cape_roof_condition_rating_severe >= 0.2
                              or cape_roof_condition_rating_poor >= 0.65)):
            if 'tarp' in cape_roof_condition_reasons or (
                ('roof_material_degradation' in cape_roof_condition_reasons) and
                (('missing_or_peeling_material'in cape_roof_material_degradation_reason) or
                 ('rusting' in cape_roof_material_degradation_reason) or
                 (cape_roof_material_degradation == "roof_material_degration_major"))):
                return 'risk'
            else:
                return 'low_risk'
        return 'low_risk'

    def fit(self, X: pd.DataFrame, y=None) -> ModelState:
        # self._declare_my_classes(self.taxonomy_labels)
        return ModelState()

    def _predict_one(self, row):
        return self.underwriting_flag_rules(row["cape_roof_condition_rating"],
                                       row["cape_roof_condition_rating_severe"],
                                       row['cape_roof_condition_rating_poor'],
                                       row['cape_roof_condition_reasons'],
                                       row['cape_roof_material_degradation_reason'],
                                       row['cape_roof_material_degradation'],
                                      )

    def predict(self, X: pd.DataFrame) -> list:
        return X.apply(self._predict_one, axis=1)

In [93]:
fs = FeatureSelect(
    [
        "cape_roof_condition_rating",  
        "cape_roof_condition_rating_severe",
        "cape_roof_condition_rating_poor",
        "cape_roof_condition_reasons",
        "cape_roof_material_degradation_reason",
        "cape_roof_material_degradation",
        "ATTOM.all_data.propertyusestandardized"
    ]
)

model = UnderwritingRules(taxonomy_labels=["risk", "low_risk"])

pipeline = CapePipeline(
    steps=[("feature_select", fs), 
           ("model", model)],
)

In [94]:
dummy_data = pd.DataFrame({
    "cape_roof_condition_rating": ['-2'],
    "cape_roof_condition_rating_severe": [0.5],
    "cape_roof_condition_rating_poor": [0.5],
    "cape_roof_condition_reasons": [["roof_material_degradation"]],
    "cape_roof_material_degradation_reason": [['rusting']],
    "cape_roof_material_degradation": ["roof_material_degration_major"],
    "ATTOM.all_data.propertyusestandardized": [123]
})
dummy_data.iloc[0]

cape_roof_condition_rating                                           -2
cape_roof_condition_rating_severe                                   0.5
cape_roof_condition_rating_poor                                     0.5
cape_roof_condition_reasons                 [roof_material_degradation]
cape_roof_material_degradation_reason                         [rusting]
cape_roof_material_degradation            roof_material_degration_major
ATTOM.all_data.propertyusestandardized                              123
Name: 0, dtype: object

In [95]:
# fit model
_ = pipeline.fit(X=dummy_data)

In [96]:
# predict
pipeline.predict(X=dummy_data)

0    risk
dtype: object